<a href="https://colab.research.google.com/github/Futaba-Kosuke/STL10/blob/feature%2Ftrain/training/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
 !ls

drive  images  sample_data


In [45]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

# バージョン確認 (Google Colab default)
print(torch.__version__)  # 1.4.0
print(torchvision.__version__)  # 0.5.0
print(np.__version__)  # 1.18.2
print(matplotlib.__version__)  # 3.2.1
print(Image.__version__)  # 7.0.0

1.4.0
0.5.0
1.18.2
3.2.1
7.0.0


In [46]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
batch_size = 50
num_workers = 3

In [48]:
# データの読み込み
transform = {
  'train': transforms.Compose(
    [transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
  ),
  'test': transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
  )
}
data_set = {
  x: torchvision.datasets.STL10(root='./images', split=x, download=True, transform=transform[x])
  for x in ('train', 'test')
}
data_size = {
  x: len(data_set[x]) for x in ('train', 'test')
}
print(data_size)

data_loaders = {
  x[0]: torch.utils.data.DataLoader(data_set[x[0]], batch_size=x[1], shuffle=x[2], num_workers=num_workers)
  for x in (('train', batch_size, True), ('test', 100, False))
}

Files already downloaded and verified
Files already downloaded and verified
{'train': 5000, 'test': 8000}


In [0]:
classes = ('airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')

In [50]:
# モデル構築
from torchvision import models
from torch import nn

net = models.vgg16(pretrained=True)
last_in_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(last_in_features, 10)  # 最終層を10クラスに変更
net = net.to(device)
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:
# 損失・最適関数の定義
from torch import optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  # 10エポックごとに学習率が1/10に更新

In [0]:
# 学習用関数
def train_net(net, criterion, optimizer, scheduler, num_epochs):
  for epoch in range(num_epochs):
    print('Epoch %d/%d' % (epoch, num_epochs - 1))
    print('-' * 10)

    running_loss_sum = 0
    train_loss_sum = 0
    test_loss_sum = 0

    # モデルの更新
    net.train()
    for i, (inputs, labels) in enumerate(data_loaders['train']):
      inputs = inputs.to(device)
      labels = labels.to(device)
      # 勾配の初期化
      optimizer.zero_grad()
      # 予測
      outputs = net(inputs)
      # 損失の導出
      loss = criterion(outputs, labels)
      # 逆伝播
      loss.backward()
      # 勾配の更新
      optimizer.step()

      running_loss_sum += loss.item()
      train_loss_sum += loss.item()
      if i % 50 == 49:
        print('[%d] running_loss: %.6f' % (i + 1, running_loss_sum / 50))
        running_loss_sum = 0

    train_cnt = i

    # モデルの評価
    net.eval()
    cnt_correct = 0
    for i, (inputs, labels) in enumerate(data_loaders['test']):
      inputs = inputs.to(device)
      labels = labels.to(device)

      # 予測
      outputs = net(inputs)
      # 損失の導出
      loss = criterion(outputs, labels)
      # lossの加算
      test_loss_sum += loss.item()

      _, predicted = torch.max(outputs, 1)
      is_correct = (predicted == labels).squeeze()
      
      for j in range(len(is_correct)):
        cnt_correct += is_correct[j].item()  # 正解なら1, 不正解なら0

    test_cnt = i

    print('')
    print('train_loss_ave:\t%.6f' % (train_loss_sum / train_cnt))
    print('test_loss_ave:\t%.6f' % (test_loss_sum / test_cnt))
    print('test_accuracy: \t%.6f %%' % (cnt_correct / (test_cnt)))

    net.train()
    scheduler.step()

    if cnt_correct / (test_cnt) > 96:
      break

In [53]:
num_epochs = 100
train_net(net, criterion, optimizer, scheduler, num_epochs)

Epoch 0/99
----------
[50] running_loss: 0.958736
[100] running_loss: 0.485877

train_loss_ave:	0.729602
test_loss_ave:	0.270298
test_accuracy: 	92.202532 %
Epoch 1/99
----------
[50] running_loss: 0.346396
[100] running_loss: 0.342959

train_loss_ave:	0.348159
test_loss_ave:	0.256832
test_accuracy: 	92.189873 %
Epoch 2/99
----------
[50] running_loss: 0.251589
[100] running_loss: 0.244123

train_loss_ave:	0.250360
test_loss_ave:	0.235185
test_accuracy: 	93.430380 %
Epoch 3/99
----------
[50] running_loss: 0.225560
[100] running_loss: 0.215524

train_loss_ave:	0.222770
test_loss_ave:	0.231747
test_accuracy: 	93.531646 %
Epoch 4/99
----------
[50] running_loss: 0.209272
[100] running_loss: 0.181720

train_loss_ave:	0.197471
test_loss_ave:	0.208702
test_accuracy: 	94.202532 %
Epoch 5/99
----------
[50] running_loss: 0.158635
[100] running_loss: 0.153279

train_loss_ave:	0.157532
test_loss_ave:	0.207469
test_accuracy: 	94.632911 %
Epoch 6/99
----------
[50] running_loss: 0.124429
[100] ru

In [0]:
# 保存
PATH = './drive/My Drive/stl_net.pth'
torch.save(net.state_dict(), PATH)

In [55]:
net.eval()

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for (inputs, labels) in data_loaders['test']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = net(inputs)
    
    _, predicted = torch.max(outputs, 1)
    is_correct = (predicted == labels).squeeze()

    for i in range(len(is_correct)):
      label = labels[i]
      class_correct[label] += is_correct[i].item()  # 正解なら1, 不正解なら0
      class_total[label] += 1

for i in range(10):
  print('Accuracy of %5s : %2.1d %%' % (
    classes[i], 100 * class_correct[i] / class_total[i]))
  
print('Accuracy Ave: %2.1d %%' % (100 * sum(class_correct) / sum(class_total)))

Accuracy of airplane : 97 %
Accuracy of  bird : 95 %
Accuracy of   car : 96 %
Accuracy of   cat : 90 %
Accuracy of  deer : 94 %
Accuracy of   dog : 91 %
Accuracy of horse : 94 %
Accuracy of monkey : 96 %
Accuracy of  ship : 98 %
Accuracy of truck : 95 %
Accuracy Ave: 94 %


In [56]:
net.eval()

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for (inputs, labels) in data_loaders['train']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = net(inputs)
    
    _, predicted = torch.max(outputs, 1)
    is_correct = (predicted == labels).squeeze()

    for i in range(len(is_correct)):
      label = labels[i]
      class_correct[label] += is_correct[i].item()  # 正解なら1, 不正解なら0
      class_total[label] += 1

for i in range(10):
  print('Accuracy of %5s : %2.1d %%' % (
    classes[i], 100 * class_correct[i] / class_total[i]))
  
print('Accuracy Ave: %2.1d %%' % (100 * sum(class_correct) / sum(class_total)))

Accuracy of airplane : 99 %
Accuracy of  bird : 99 %
Accuracy of   car : 98 %
Accuracy of   cat : 98 %
Accuracy of  deer : 99 %
Accuracy of   dog : 98 %
Accuracy of horse : 99 %
Accuracy of monkey : 99 %
Accuracy of  ship : 100 %
Accuracy of truck : 99 %
Accuracy Ave: 99 %
